In [31]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import make_scorer,mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV


In [47]:
data=pd.read_csv('X_train1.csv')
test=pd.read_csv('X_test.csv')



In [56]:
data.shape



(8523, 18)

In [49]:
data.head()

,Item_Identifier,Item_Outlet_Sales,Item_Weight,Item_Visibility,Item_MRP,Years active,Item_Type_Combined,Outlet_No,Medium,Small,U,Tier 2,Tier 3,Supermarket Type1,Supermarket Type2,Supermarket Type3,Non-Edible,Regular
0,FDA15,8.225540,0.462051,0.332202,0.969148,0.416667,0.5,1.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,DRC01,6.094524,0.169670,0.372782,0.202234,0.000000,0.0,0.205128,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,FDN15,7.648392,0.871279,0.341816,0.704368,0.416667,0.5,1.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,FDX07,6.596300,0.931292,0.600360,0.821649,0.458333,0.5,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,NCD19,6.902446,0.435771,0.600360,0.253374,0.916667,1.0,0.076923,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0


In [50]:
test.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_No,Years active,Item_Type_Combined,Medium,Small,U,Tier 2,Tier 3,Supermarket Type1,Supermarket Type2,Supermarket Type3,Non-Edible,Regular
0,FDW58,0.981548,0.165508,0.573240,1.000000,0.416667,0.5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,FDW14,0.388413,0.526592,0.473594,0.179487,0.083333,0.5,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,NCN55,0.753998,0.738126,0.953881,0.000000,0.458333,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,FDQ58,0.306637,0.323271,0.744345,0.179487,0.083333,0.5,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,FDY38,0.653474,0.776971,0.938969,0.435897,1.000000,0.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [57]:
X=data.drop(['Item_Identifier','Item_Outlet_Sales'],axis=1)
y=data['Item_Outlet_Sales']
X_test=test.drop(['Item_Identifier'],axis=1)

In [63]:
X_test

,Item_Weight,Item_Visibility,Item_MRP,Outlet_No,Years active,Item_Type_Combined,Medium,Small,U,Tier 2,Tier 3,Supermarket Type1,Supermarket Type2,Supermarket Type3,Non-Edible,Regular
0,0.981548,0.165508,0.573240,1.000000,0.416667,0.5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.388413,0.526592,0.473594,0.179487,0.083333,0.5,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.753998,0.738126,0.953881,0.000000,0.458333,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,0.306637,0.323271,0.744345,0.179487,0.083333,0.5,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.653474,0.776971,0.938969,0.435897,1.000000,0.5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5676,0.540611,0.294126,0.700648,0.923077,0.500000,0.5,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5677,0.331379,0.818523,0.785429,0.205128,0.000000,0.5,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
5678,0.509028,0.670757,0.618572,0.897436,0.291667,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
5679,0.784313,0.602809,0.897736,0.179487,0.083333,0.5,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0


In [64]:
X

,Item_Weight,Item_Visibility,Item_MRP,Years active,Item_Type_Combined,Outlet_No,Medium,Small,U,Tier 2,Tier 3,Supermarket Type1,Supermarket Type2,Supermarket Type3,Non-Edible,Regular
0,0.462051,0.332202,0.969148,0.416667,0.5,1.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.169670,0.372782,0.202234,0.000000,0.0,0.205128,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,0.871279,0.341816,0.704368,0.416667,0.5,1.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.931292,0.600360,0.821649,0.458333,0.5,0.000000,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,0.435771,0.600360,0.253374,0.916667,1.0,0.076923,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,0.265538,0.611762,0.898103,0.916667,0.5,0.076923,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
8519,0.394622,0.569847,0.578618,0.291667,0.5,0.897436,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
8520,0.546747,0.505887,0.466889,0.208333,1.0,0.641026,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0
8521,0.297278,0.819493,0.556429,0.000000,0.5,0.205128,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [59]:
def test_model(model,X,y):
  cv=KFold(n_splits=3,shuffle=True,random_state=45)
  r2_val_score=cross_val_score(model,X,y,cv=cv,error_score=mean_squared_error)
  score=[r2_val_score.mean()]
  return score

In [60]:
model1=LinearRegression()
model1.fit(X,y)
test_model(model1,X,y)
predict1=model1.predict(X_test)


In [68]:
model2=XGBRegressor()
model2.fit(X,y)
test_model(model2,X,y)




[16:55:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:55:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:55:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:55:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


ValueError: ignored

In [67]:
model3=SVR(kernel='linear')
model3.fit(X,y)
test_model(model3,X,y)
predict3=model3.predict(X_test)



In [11]:
model4=DecisionTreeRegressor()
test_model(model4,X,y)

[0.4390509977224905]

In [12]:
model5=RandomForestRegressor()
test_model(model5,X,y)

[0.7018546241507514]

In [13]:
#hyperparametertuning
def hyperparamter_tuning(model,parameters):
  rst_search=RandomizedSearchCV(model,param_distributions=parameters,n_iter=10,n_jobs=-1,cv=10)
  rst_search.fit(X,y)
  high_score=rst_search.best_score_
  best_paramters=rst_search.best_params_

  return high_score,best_paramters





In [14]:
parameter_1={'max_depth':[1,2,3,4,5],'n_estimators':[100,200,300,400,500],'learning_rate':[0.1,0.2,0.3,0.4,0.5],'gamma':[0,0.1,0.2,0.3,0.4,0.5],'min_child_weight':[1,2,3,4,5,6]}

In [15]:
hyperparamter_tuning(model2,parameter_1)

[13:39:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(0.7368980505814717,
 {'gamma': 0.4,
  'learning_rate': 0.1,
  'max_depth': 2,
  'min_child_weight': 5,
  'n_estimators': 300})

In [ ]:
SVR_predict=pd.concat([test['Item_Identifier'],data1['Outlet_Identifier'],pd.DataFrame(predict3)],axis=1)
SVR_predict.columns=['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']
XG_predict=pd.concat([test['Item_Identifier'],data1['Outlet_Identifier'],pd.DataFrame(predict2)],axis=1)
XG_predict.columns=['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']
Linear_predict=pd.concat([test['Item_Identifier'],data1['Outlet_Identifier'],pd.DataFrame(predict1)],axis=1)
Linear_predict.columns=['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']

In [ ]:
Linear_predict

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1580.898835
1,FDW14,OUT017,1238.941603
2,NCN55,OUT010,496.283220
3,FDQ58,OUT017,2303.556916
4,FDY38,OUT027,5921.456189
...,...,...,...
5676,FDB58,OUT046,2046.509145
5677,FDD47,OUT018,2089.365695
5678,NCO17,OUT045,1620.980986
5679,FDJ26,OUT017,3270.361712


In [ ]:
SVR_predict.to_csv('SVC.csv',index=False)
Linear_predict.to_csv('Linear.csv',index=False)
